In [6]:
from src.helpers import json_to_owl, parse_rdf_constraints, extract_relevant_constraints
import json
import re
from rdflib import Graph, Namespace, Literal, RDF, RDFS, URIRef
from rdflib.namespace import XSD, OWL
import datetime
from logs_simulation.simulation_process import *
import sqlite3
import csv

with open("visuals/HSL248.json", "r", encoding="utf-8") as f:
    file_json = json.load(f)

graph = json_to_owl(file_json, namespace="urn:crawlercrane-ontology#")

output_file = "ontologies/crawler_crane_ontology.ttl"

with open(output_file, "w", encoding="utf-8") as out_file:
    out_file.write(graph.serialize(format="turtle"))

In [7]:
g = Graph()
output_file = "ontologies/crawler_crane_ontology.ttl"
with open(output_file, "w", encoding="utf-8") as out_file:
    out_file.write(graph.serialize(format="turtle"))
g.parse("ontologies/crawler_crane_ontology.ttl", format="turtle")

<Graph identifier=Nb5b3a6d09eb64f159246d0b5eeab348f (<class 'rdflib.graph.Graph'>)>

In [8]:
rdf_constraints = parse_rdf_constraints(output_file)

In [9]:
component_sensor_definitions = {}
for comp_iri, constraints in rdf_constraints.items():
    sensor_defs = extract_relevant_constraints(constraints)
    component_sensor_definitions[comp_iri] = sensor_defs

In [10]:
num_machines = 10
rows_per_machine = 1000
total_rows = num_machines * rows_per_machine

all_components = list(rdf_constraints.keys())

start_time = datetime.datetime.now()

connection = sqlite3.connect("logs_simulation/machine_data.db")
cursor = connection.cursor()
row_count = 0
for machine_id in range(1, num_machines + 1):
    for i in range(rows_per_machine):
        # 1) Pick a random component
        component_iri = random.choice(all_components)
        comp_sensors = component_sensor_definitions[component_iri]

        # If no extracted sensors, skip this row or produce a dummy sensor
        if not comp_sensors:
            # Insert a dummy sensor reading to show everything has "some telemetry"
            sensor_name = "misc_sensor"
            sensor_value = random.uniform(0, 1)
            sensor_unit = "N/A"
            dt_str = random_timestamp(start_time, row_count)

            cursor.execute(
                """
                INSERT INTO telemetry (timestamp, machine_id, component, sensor_name, sensor_value, sensor_unit)
                VALUES (?, ?, ?, ?, ?, ?)
            """,
                (dt_str, machine_id, component_iri, sensor_name, sensor_value, sensor_unit),
            )

        else:
            sensor_tuple = random.choice(comp_sensors)
            (raw_key, sensor_name, max_val, min_val) = sensor_tuple

            dt_str = random_timestamp(start_time, row_count)
            sensor_value = generate_sensor_value(sensor_tuple)

            # The original RDF says e.g. 'unit': 'm' or 'rpm'. Attempt to retrieve:
            sensor_unit = rdf_constraints[component_iri][raw_key].get("unit", "N/A")

            cursor.execute(
                """
                INSERT INTO telemetry (timestamp, machine_id, component, sensor_name, sensor_value, sensor_unit)
                VALUES (?, ?, ?, ?, ?, ?)
            """,
                (dt_str, machine_id, component_iri, sensor_name, sensor_value, sensor_unit),
            )

        row_count += 1

connection.commit()


# -------------------------------------------------------
# 5) Show some rows from the in-memory table
# -------------------------------------------------------
cursor.execute("SELECT * FROM telemetry LIMIT 20")
rows = cursor.fetchall()
for r in rows:
    print(r)

print(f"Total rows inserted: {row_count}")


# import to csv file for further references
with open("logs_simulation/logs/dummy_telemetry.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([desc[0] for desc in cursor.description])  # column headers
    writer.writerows(cursor.execute("SELECT * FROM telemetry"))

# Done!
connection.close()

(1, '2025-01-12 15:07:14', 1, 'urn:crawlercrane-ontology#Swing_System', 'swingspeed', 0.7255944417257613, 'rpm')
(2, '2025-01-12 15:08:14', 1, 'urn:crawlercrane-ontology#Luffing_Jib', 'luffingjiblength', 21.636292554415622, 'm')
(3, '2025-01-12 15:09:14', 1, 'urn:crawlercrane-ontology#Hydraulic_System', 'hydraulicpressure', 205.07369264538053, 'bar')
(4, '2025-01-12 15:10:14', 1, 'urn:crawlercrane-ontology#Load_Hoist_Drums', 'wireropediameter', 19.092855081801876, 'mm')
(5, '2025-01-12 15:11:14', 1, 'urn:crawlercrane-ontology#Luffing_Boom', 'luffingboomlength', 16.12588575204922, 'm')
(6, '2025-01-12 15:12:14', 1, 'urn:crawlercrane-ontology#Rated_Capacity_Limiter_System', 'misc_sensor', 0.7065892272708995, 'N/A')
(7, '2025-01-12 15:13:14', 1, 'urn:crawlercrane-ontology#Counterweight', 'counterweight', 6.6535313244765995, '470 kg')
(8, '2025-01-12 15:14:14', 1, 'urn:crawlercrane-ontology#Fuel_Tank', 'fueltanklevel', 223.8931618783332, 'L')
(9, '2025-01-12 15:15:14', 1, 'urn:crawlercrane

In [13]:
connection = sqlite3.connect("logs_simulation/machine_data.db")
cursor = connection.cursor()
cursor.execute("SELECT * FROM telemetry where machine_id=2 limit 5")
rows = cursor.fetchall()
for r in rows:
    print(r)

print(f"Total rows inserted: {row_count}")
connection.close()

(1001, '2025-01-13 07:47:14', 2, 'urn:crawlercrane-ontology#Fuel_Tank', 'fueltanklevel', 397.13590231176295, 'L')
(1002, '2025-01-13 07:48:14', 2, 'urn:crawlercrane-ontology#Tubular_Jib', 'jiblength', 21.88587568189134, 'm')
(1003, '2025-01-13 07:49:14', 2, 'urn:crawlercrane-ontology#Conventional_Tube_Boom', 'boomlength', 50.43618989831721, 'm')
(1004, '2025-01-13 07:50:14', 2, 'urn:crawlercrane-ontology#Load_Hoist_Drums', 'wireropediameter', 19.69688312114644, 'mm')
(1005, '2025-01-13 07:51:14', 2, 'urn:crawlercrane-ontology#Hydraulic_Pumps', 'flowrate', 124.86378090331152, 'L/min')
Total rows inserted: 10000
